In [19]:
import pandas as pd
import numpy as np
import datetime as dt

In [20]:
OBD = pd.read_excel(r'C:\Users\prajwal.d1\Desktop\PythonCheck\OBDW10.xlsx', engine="openpyxl", sheet_name="Sheet1")
CV = pd.read_excel(r'C:\Users\prajwal.d1\Desktop\PythonCheck\CVW1510A.xlsx', engine="openpyxl", sheet_name="Sheet1")
PRC = pd.read_excel(r'C:\Users\prajwal.d1\Desktop\PythonCheck\PRC30MAR.xlsx', engine="openpyxl", sheet_name="Hub Wise") 
GTNLPendency = pd.read_excel(r'C:\Users\prajwal.d1\Desktop\PythonCheck\gtnl_pendency8Apr.xlsx', engine="openpyxl", sheet_name="Sheet1")
GTNLCVAdhernce = pd.read_excel(r'C:\Users\prajwal.d1\Desktop\PythonCheck\GTNLCV2A8A.xlsx', engine="openpyxl", sheet_name="Raw")
Map = pd.read_csv(r'C:\Users\prajwal.d1\Desktop\PythonCheck\ODH Model pincode Tracker.csv')
Relabel  = pd.read_csv(r'C:\Users\prajwal.d1\Desktop\PythonCheck\Relabel.csv')  #Done
LogPendency  = pd.read_excel(r'C:\Users\prajwal.d1\Desktop\PythonCheck\G_M_Pendency_data 2025-04-11.xlsx', engine="openpyxl", sheet_name="Sheet1")

In [21]:
# Relabel 

Relabel = Relabel[['Hub_name', 'Vendor Name ', 'relabel', 'OFD']]
Relabel = Relabel.dropna(subset=['Vendor Name '])
Relabel.to_csv("C:/Users/prajwal.d1/Desktop/PythonCheck/Relabel.csv")


In [22]:
# PRC 

PRC = PRC[['Vendor Name ', 'Hub Name','Zone','Sum of Final Reco', 'Sum of Non Adherence','Sum of Excess Shipment', 'Grand Total', 'Final Reco Completed %']]
PRC.rename(columns={
    'Vendor Name ': 'Vendor Name',
    'Hub Name': 'Hub Name',
    'Zone': 'Zone',
    'Sum of Final Reco': 'Final Reco',
    'Sum of Non Adherence': 'Non Adherence',
    'Sum of Excess Shipment': 'Excess Shipment',
    'Grand Total': 'Grand Total',
    'Final Reco Completed %': 'Final Reco Completed %'
}, inplace=True)
PRC = PRC.dropna(subset=['Vendor Name'])
PRC.to_csv("C:/Users/prajwal.d1/Desktop/PythonCheck/PRC.csv")


In [23]:
#OBD Done 

OBD = OBD[['Vendor Name', 'parent_lm_hub', 'Image Adherence', 'tracking_id']]
OBD_pivot = pd.pivot_table(
    OBD,
    values='tracking_id',
    index=['Vendor Name', 'parent_lm_hub'],
    columns='Image Adherence',
    aggfunc='count',
    fill_value=0  
)
OBD_pivot['Grand Total'] = OBD_pivot.sum(axis=1)
OBD_pivot['Adherence'] = OBD_pivot['Okay'] / OBD_pivot['Grand Total'] *100
OBD_pivot['Adherence'] = OBD_pivot['Adherence'].round(2).astype(str) + '%'
OBD_pivot.to_csv("C:/Users/prajwal.d1/Desktop/PythonCheck/OBD_pivot.csv")

In [24]:
#CVDone

CV_pivot = pd.pivot_table(
    CV,
    values='ShipmentId',
    index=['Bucket', 'Connected Check', 'Type', 'Date','Vendor Name','CurrentHubName'],
    columns='CV Check',
    aggfunc='count',
    fill_value=0  
)
CV_pivot['Grand Total'] = CV_pivot.sum(axis=1)
CV_pivot.to_csv("C:/Users/prajwal.d1/Desktop/PythonCheck/CV_pivot.csv")

In [25]:
#GTNL Pendecy 

GTNLPendency = GTNLPendency[~GTNLPendency['Status'].isin(['Expected', 'Lost', 'Untraceable_BRSNR'])]


GTNL_pivot = pd.pivot_table(
    GTNLPendency,
    values=['shipment_id', 'return_amount'],
    index=['Bucket', 'Age Bucket', 'Vendor Name', 'HubName'],
    aggfunc={'shipment_id': 'count', 'return_amount': 'sum'},
    fill_value=0
)
GTNL_pivot.to_csv("C:/Users/prajwal.d1/Desktop/PythonCheck/GTNL_pivot.csv")

In [26]:
#Log Pendency 

LogPendency = LogPendency[LogPendency["New Remarks"].isin(["DH Pendency"])]
LogPendency = pd.pivot_table(
    LogPendency,
    values=['ShipmentId', ' TotalPrice'],
    index=['Bucket', 'Slab', 'New BagStatus','Vendor', 'Mapped hub'],
    aggfunc={'ShipmentId': 'count', ' TotalPrice': 'sum'},
    fill_value=0
)
LogPendency.to_csv("C:/Users/prajwal.d1/Desktop/PythonCheck/LogPendency.csv")

In [ ]:

Map = Map[['ODH/MDH Name','Vendor']]
Map['Vendor'] = Map['Vendor'].astype(str).str.upper()
Map['ODH/MDH Name'] = Map['ODH/MDH Name'].astype(str).str.upper()


In [ ]:
GTNLCVAdhernce = pd.merge(GTNLCVAdhernce, Map, how="left", left_on="DH", right_on="ODH/MDH Name")
GTNLCVAdhernce = GTNLCVAdhernce.dropna(subset = 'Vendor', how = 'all')
GTNLCVAdhernce

In [ ]:
GTNLCVAdhernce = GTNLCVAdhernce[~GTNLCVAdhernce['Latest Status'].isin(['Delivery_Update', 'Error','InScan_Success','Out_For_Delivery','Untraceable_BRSNR',''])]
GTNLCVAdhernce = GTNLCVAdhernce[~GTNLCVAdhernce['rtostageattributed'].isin(['RTO - AT MH/TC OR INTRANSIT', 'RTO - AT PPMH',''])]
GTNLCVAdhernce = GTNLCVAdhernce[~GTNLCVAdhernce['rtostageattributed'].isin(['Non-Serviceable',''])]


In [27]:
#GTNL CV 

GTNLCVAdhernce=GTNLCVAdhernce [['Vendor Name','DH','CV Done ','tracking_id']]
GTNLCVAdhernce = pd.pivot_table(
    GTNLCVAdhernce,                      # source DataFrame
    values=['tracking_id'],             # column to aggregate
    index=['Vendor Name', 'DH'],        # group by these columns
    columns=['CV Done '],               # separate columns by 'CV Done ' status
    aggfunc={'tracking_id': 'count'},   # count number of tracking_ids
    fill_value=0                        # fill empty cells with 0
)

GTNLCVAdhernce.to_csv("C:/Users/prajwal.d1/Desktop/PythonCheck/GTNLCVAdhernce.csv")


In [28]:
# MERGE ALL 

Relabel = pd.read_csv(r'C:\Users\prajwal.d1\Desktop\PythonCheck\Relabel.csv', dtype=str)
PRC = pd.read_csv(r'C:\Users\prajwal.d1\Desktop\PythonCheck\PRC.csv', dtype=str)
OBD_pivot = pd.read_csv(r'C:\Users\prajwal.d1\Desktop\PythonCheck\OBD_pivot.csv', dtype=str)
CV_pivot = pd.read_csv(r'C:\Users\prajwal.d1\Desktop\PythonCheck\CV_pivot.csv', dtype=str)
GTNL_pivot = pd.read_csv(r'C:\Users\prajwal.d1\Desktop\PythonCheck\GTNL_pivot.csv', dtype=str)
LogPendency = pd.read_csv(r'C:\Users\prajwal.d1\Desktop\PythonCheck\LogPendency.csv', dtype=str)
GTNLCVAdhernce = pd.read_csv(r'C:\Users\prajwal.d1\Desktop\PythonCheck\GTNLCVAdhernce.csv', dtype=str)


In [29]:
#OutPut File 

output_file = 'C:/Users/prajwal.d1/Desktop/PythonCheck/PowerBI_OP.xlsx'

with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    
    Relabel.to_excel(writer, sheet_name='Relabel', index=False)
    PRC.to_excel(writer, sheet_name='PRC', index=False)
    OBD_pivot.to_excel(writer, sheet_name='OBD_pivot', index=False)
    CV_pivot.to_excel(writer, sheet_name='CV_pivot', index=False)
    GTNL_pivot.to_excel(writer, sheet_name='GTNL_pivot', index=False)
    LogPendency.to_excel(writer, sheet_name='LogPendency', index=False)
    GTNLCVAdhernce.to_excel(writer, sheet_name='GTNLCVAdhernce', index=False)